In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip /kaggle/input/dogs-vs-cats/train.zip

In [ ]:
imgs = os.listdir("./train")

In [ ]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset , DataLoader
from torchvision import transforms
from torch.optim import Adam
from torch.autograd import Variable

In [ ]:
train_imgs = np.random.choice(imgs , 20000 , replace = False)

In [ ]:
test_imgs = np.setdiff1d(imgs , train_imgs)

In [ ]:
class_to_int = {"cat" : 0 , "dog" : 1}
train_transforms = transforms.Compose([
    transforms.Resize((360 , 360)) , 
    transforms.RandomHorizontalFlip(p = 0.5) , 
    transforms.ToTensor() , 
    transforms.Normalize((0 , 0 , 0) , (1 , 1 , 1))
])

test_pred_transforms = transforms.Compose([
    transforms.Resize((360 , 360)) , 
    transforms.ToTensor() , 
    transforms.Normalize((0 , 0 , 0) , (1 , 1 , 1))
])

In [ ]:
class CustDat(Dataset):
    def __init__(self ,imgs , class_to_int , path , mode , transforms):
        self.imgs = imgs
        self.class_to_int = class_to_int
        self.mode = mode
        self.transforms = transforms
        self.path = path
        
    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self , idx):
        img = Image.open(self.path + self.imgs[idx])
        if self.mode  in ["train" , "test"]:
            label = self.class_to_int[self.imgs[idx].split(".")[0]]
            img = self.transforms(img)
            return img , label
        elif self.mode == "pred":
            img = self.transforms(img)
            return img

In [ ]:
train = DataLoader(CustDat(train_imgs , class_to_int , "./train/" , "train" , train_transforms) , batch_size = 20 , shuffle = True)
test = DataLoader(CustDat(test_imgs , class_to_int , "./train/" , "test" , test_pred_transforms) , batch_size = 20 , shuffle = True)

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

In [ ]:
num_last_features = model.fc.in_features

In [ ]:
new_last_yr = nn.Sequential(
nn.Linear(num_last_features , 1) , 
    nn.Sigmoid()
)

In [ ]:
model.fc = new_last_yr

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model_fin = model.to(device)

In [ ]:
criterion = nn.BCELoss()
optimizer = Adam(model_fin.fc.parameters() , lr = 0.001)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model_fin.train()
    su_tr = 0
    tot_tr = 0
    loss_list = []
    for images , labels in train:
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
        optimizer.zero_grad()
        output = model_fin(images)
        loss = criterion(torch.squeeze(output) , labels.float())
        loss_list.append(loss.data)
        su_tr += torch.sum(output.reshape(labels.shape).round() == labels)
        tot_tr += labels.shape[0]
        loss.backward()
        optimizer.step()
    model_fin.eval()
    su_te = 0
    tot_te = 0
    with torch.no_grad():
        for images , labels in test:
            output = model_fin(images.cuda())
            su_te += torch.sum(output.reshape(labels.shape).round().cpu() == labels)
            tot_te += labels.shape[0]
    print("loss is "+str(torch.mean(torch.tensor(loss_list)))+" train accu "+str(su_tr/tot_tr)+" test accu "+str(su_te/tot_te))